In [1]:
%%capture
pip install git+https://github.com/boudinfl/pke.git 

In [2]:
%%capture
#pip install flashtext

In [3]:
#pip install --upgrade spacy==2.2.4
#pip show spacy

### Keyword Extractor Module
   1. The process of extraction of keywords from a text is an unsupervised task. It can be done using  **Statistical models**  and **Graph-based models** 
   2. PKE unsupervised library provides us with 3 statistical models and 6 graph based models.
   3. The notebook compares the performances of all the well known models for the given input text.


### References
   1. https://towardsdatascience.com/keyword-extraction-python-tf-idf-textrank-topicrank-yake-bert-7405d51cd839
   2. https://boudinfl.github.io/pke/build/html/unsupervised.html
   3. https://arxiv.org/abs/1803.08721
   4. https://github.com/MaartenGr/KeyBERT#usage
    


### Setup and Data collection

In [4]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
from keybert import KeyBERT

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16692\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\16692\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\16692\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# This is the sample input text
text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""
len(text.split())

215

Name: spacy
Version: 2.2.4
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: c:\users\16692\anaconda3\lib\site-packages
Requires: murmurhash, blis, setuptools, numpy, requests, cymem, preshed, plac, wasabi, srsly, catalogue, thinc, tqdm
Required-by: pke, en-core-web-sm
Note: you may need to restart the kernel to use updated packages.


In [16]:
def get_keyword_limit(no_of_words,user_limit = 0):
    if user_limit ==0:
        return round(no_of_words/15)


In [28]:
def get_keywords_MultipartiteRank(text):
    """Extracts keywords from input text using Graph based MultipartiteRank algorithm
           @input: (text)-Block of continous text on a single topic 
           @Hyperparameters: ()-None
           @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out={}
    try:
        # Selecting the extractor and loading the input text in it
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=text)
        
        # It can extract these three types of "phrases" from the input text
        pos = {'VERB', 'ADJ', 'NOUN'}
        convert_pos = {'VERB': 'v', 'NOUN': 'n','ADJ': 'a'}
        
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        
        keyword_limit = get_keyword_limit(len(text.split()))
        
        for partsofspeech in pos:
            extractor.candidate_selection(pos=partsofspeech, stoplist=stoplist)
            extractor.candidate_weighting(alpha=1.1, threshold=0.75, method='average')
            keyphrases = extractor.get_n_best(n=keyword_limit)
            
            for val in keyphrases:
                out[(val[0],convert_pos[partsofspeech])] = val[1]
                
    except:
        out = {}
        traceback.print_exc()

    return out

In [29]:
k = get_keywords_MultipartiteRank(text)
print(len(k))
print(k)

42
{('oceans', 'n'): 0.10302416744017867, ('undersea volcanoes', 'n'): 0.0884645405518853, ('plate boundaries', 'n'): 0.08432931637295996, ('plate', 'n'): 0.07096028545990593, ('rock', 'n'): 0.058991754010652334, ('crust', 'n'): 0.053535846868880216, ('example', 'n'): 0.051820409215493114, ('cracks', 'n'): 0.03541664826727955, ('activity', 'n'): 0.035349980756700164, ('surface', 'n'): 0.033796845675054095, ('subduction zone', 'n'): 0.033354679877531386, ('volcanoes', 'n'): 0.030537514820040943, ('lot', 'n'): 0.028607846755605654, ('volcanoes form', 'n'): 0.028531446593391147, ('oceans', 'a'): 0.07640427896367138, ('plate boundaries', 'a'): 0.07453170589051847, ('divergent', 'a'): 0.061340552016611725, ('plate', 'a'): 0.05940852072369033, ('undersea volcanoes', 'a'): 0.05789660999344903, ('rock', 'a'): 0.044473208150754846, ('crust', 'a'): 0.043202585567860684, ('many', 'a'): 0.04081820498072136, ('molten', 'a'): 0.03498437668426168, ('boundaries', 'a'): 0.03187732713063273, ('volcanic'

In [36]:
def get_keywords_PositionRank(text):
    """Extracts keywords from input text using Graph based PositionRank algorithm
       @input: (text)-Block of continous text on a single topic 
       @Hyperparameters: ()-None
       @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out = {}
    pos = {'NOUN', 'VERB', 'ADJ'}
    convert_pos = {'VERB': 'v', 'NOUN': 'n','ADJ': 'a'}
    try:
    # define the grammar for selecting the keyphrase candidates
        grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

        # 1. create a PositionRank extractor.
        extractor = pke.unsupervised.PositionRank()
        #print("Keyword extraction Using the PositionRank model")

        # 2. load the content of the document.
        extractor.load_document(input=text, language='en', normalization=None)

        # 3. select the noun phrases up to 3 words as keyphrase candidates.
        extractor.candidate_selection(grammar=grammar,maximum_word_number=3)

        keyword_limit = get_keyword_limit(len(text.split()))

        # 4. weight the candidates using the sum of their word's scores that are
        #    computed using random walk biaised with the position of the words
        #    in the document. In the graph, nodes are words (nouns and
        #    adjectives only) that are connected if they occur in a window of
        #    10 words.
        for partsofspeech in pos:
            extractor.candidate_weighting(window=10,pos=partsofspeech)
            # 5. get the 10-highest scored candidates as keyphrases
            keyphrases = extractor.get_n_best(n=keyword_limit)
            for val in keyphrases:
                out[(val[0],convert_pos[partsofspeech])] = val[1]
    except:
        out = {}
        traceback.print_exc()

    return out

In [37]:
k = get_keywords_PositionRank(text)
print(len(k))
print(k)

42
{('convergent plate boundaries', 'n'): 0.2740242877458622, ('divergent plate boundaries', 'n'): 0.2554064196108812, ('plate melts', 'n'): 0.16350633422275607, ('divergent plate', 'n'): 0.14872209540330936, ('tectonic plate', 'n'): 0.14872209540330936, ('pacific plate', 'n'): 0.14872209540330936, ('many undersea volcanoes', 'n'): 0.11632726026346255, ('volcanoes form', 'n'): 0.1085462034519323, ('volcanoes forms', 'n'): 0.10739561111920042, ('boundaries', 'n'): 0.10668432420757185, ('many volcanoes', 'n'): 0.09300551647238586, ('volcanoes', 'n'): 0.09300551647238586, ('ocean crust', 'n'): 0.07525084121064403, ('subduction zone', 'n'): 0.0713412828131566, ('divergent plate boundaries', 'a'): 0.24993648305141067, ('divergent plate', 'a'): 0.1805063660249963, ('convergent plate boundaries', 'a'): 0.17833335194555583, ('tectonic plate', 'a'): 0.12482526453961151, ('plate melts', 'a'): 0.10640798574365178, ('pacific plate', 'a'): 0.09673440331095517, ('many undersea volcanoes', 'a'): 0.09

In [59]:
def get_keywords_TopicRank(text):
    """Extracts keywords from input text using Graph based TopicRank algorithm
           @input: (text)-Block of continous text on a single topic 
           @Hyperparameters: ()-None
           @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out={}
    try:
        # Selecting the extractor and loading the input text in it
        extractor = pke.unsupervised.TopicRank()
        extractor.load_document(input=text)
        
        # It can extract these three types of "phrases" from the input text
        pos = {'NOUN','VERB','ADJ'}
        convert_pos = {'VERB': 'v', 'NOUN': 'n','ADJ': 'a'}
        
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        
        keyword_limit = get_keyword_limit(len(text.split()))
        
        for partsofspeech in pos:
            extractor.candidate_selection(pos=partsofspeech, stoplist=stoplist)
            extractor.candidate_weighting(threshold=0.74, method='average')
            try:
                keyphrases = extractor.get_n_best(n=keyword_limit)
            except ZeroDivisionError:
                continue
                
            
            for val in keyphrases:
                out[(val[0],convert_pos[partsofspeech])] = val[1]
                
    except:
        out = {}
        traceback.print_exc()

    return out

In [60]:
k = get_keywords_TopicRank(text)
print(len(k))
print(k)

14
{('undersea volcanoes', 'n'): 0.11166551175242785, ('plate', 'n'): 0.09315379045958856, ('crust', 'n'): 0.09304248626350621, ('plate boundaries', 'n'): 0.08217261390177774, ('rock', 'n'): 0.08039706362726223, ('subduction zone', 'n'): 0.05670403385765736, ('cracks', 'n'): 0.04873040443332037, ('surface', 'n'): 0.04765114022376231, ('edge', 'n'): 0.04213339252919163, ('earth', 'n'): 0.03292826335901927, ('lava', 'n'): 0.027723966984365897, ('hardens', 'n'): 0.027506314914308515, ('line', 'n'): 0.026729025642243043, ('oceans', 'n'): 0.026024166094772424}


In [61]:
def get_keywords_SingleRank(text):
    """Extracts keywords from input text using Graph based SingleRank algorithm
           @input: (text)-Block of continous text on a single topic 
           @Hyperparameters: ()-None
           @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out={}
    try:
        # Selecting the extractor and loading the input text in it
        extractor = pke.unsupervised.SingleRank()
        extractor.load_document(input=text)
        
        # It can extract these three types of "phrases" from the input text
        pos = {'NOUN','VERB','ADJ'}
        convert_pos = {'VERB': 'v', 'NOUN': 'n','ADJ': 'a'}
        
        
        keyword_limit = get_keyword_limit(len(text.split()))
        
        for partsofspeech in pos:
            extractor.candidate_selection(pos=partsofspeech)
            extractor.candidate_weighting(window=10,pos=partsofspeech)
            try:
                keyphrases = extractor.get_n_best(n=keyword_limit)
            except ZeroDivisionError:
                continue
                
            
            for val in keyphrases:
                out[(val[0],convert_pos[partsofspeech])] = val[1]
                
    except:
        out = {}
        traceback.print_exc()

    return out

In [62]:
k = get_keywords_SingleRank(text)
print(len(k))
print(k)

42
{('convergent plate boundaries', 'n'): 0.23303142898104937, ('plate boundaries', 'n'): 0.21164701592346064, ('plate melts', 'n'): 0.1529143597366299, ('plate', 'n'): 0.13009334726510488, ('volcanoes form', 'n'): 0.12815494942708625, ('undersea volcanoes', 'n'): 0.11271267981251763, ('subduction zone', 'n'): 0.10508687953426911, ('ocean crust', 'n'): 0.09987348313128479, ('volcanoes', 'n'): 0.0937939313158603, ('boundaries', 'n'): 0.08155514865835575, ('crust', 'n'): 0.06343458355054288, ('rock', 'n'): 0.05756125315613764, ('cracks', 'n'): 0.036838897581556417, ('oceans', 'n'): 0.03643845958074192, ('convergent plate boundaries', 'a'): 0.2003278639275302, ('plate boundaries', 'a'): 0.18362919326157334, ('plate melts', 'a'): 0.11854193599325631, ('plate', 'a'): 0.10132062811416108, ('volcanoes form', 'a'): 0.10071571298013135, ('ocean crust', 'a'): 0.09923626942619934, ('undersea volcanoes', 'a'): 0.08914418592765692, ('boundaries', 'a'): 0.08231004514741228, ('subduction zone', 'a'):

In [71]:
def get_keywords_TextRank(text):
    """Extracts keywords from input text using Graph based TextRank algorithm
           @input: (text)-Block of continous text on a single topic 
           @Hyperparameters: ()-None
           @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out={}
    try:
        # Selecting the extractor and loading the input text in it
        extractor = pke.unsupervised.TextRank()
        extractor.load_document(input=text)
        
        # It can extract these three types of "phrases" from the input text
        pos = {'NOUN','VERB','ADJ'}
        convert_pos = {'VERB': 'v', 'NOUN': 'n','ADJ': 'a'}
        
        
        keyword_limit = get_keyword_limit(len(text.split()))
        
        for partsofspeech in pos:
            #extractor.candidate_selection(pos=partsofspeech)
            extractor.candidate_weighting(window=3,pos=partsofspeech,top_percent=1)
            try:
                keyphrases = extractor.get_n_best(n=keyword_limit)
            except ZeroDivisionError:
                continue
                
            
            for val in keyphrases:
                out[(val[0],convert_pos[partsofspeech])] = val[1]
                
    except:
        out = {}
        traceback.print_exc()

    return out

In [72]:
k = get_keywords_TextRank(text)
print(len(k))
print(k)

42
{('convergent plate boundaries', 'n'): 0.20261913074467355, ('volcanoes form', 'n'): 0.17104580408252842, ('undersea volcanoes', 'n'): 0.17104480408252842, ('subduction zones lined', 'n'): 0.15515425364174637, ('plate boundaries', 'n'): 0.14458154907457962, ('volcanoes', 'n'): 0.13388874073202472, ('plate melts', 'n'): 0.11991796724229746, ('subduction zone', 'n'): 0.11799587029124269, ('ocean crust', 'n'): 0.10862324009019227, ('plates', 'n'): 0.08654592740448569, ('earth', 'n'): 0.0862973872177052, ('crust', 'n'): 0.06991635638680654, ('boundaries', 'n'): 0.05803727167009392, ('forming rock', 'n'): 0.046008490874786756, ('convergent plate boundaries', 'a'): 0.17504122795861413, ('divergent plate boundaries', 'a'): 0.15646467015122495, ('many volcanoes form', 'a'): 0.13364791310835694, ('many undersea volcanoes', 'a'): 0.13364650310835693, ('plate boundaries', 'a'): 0.13111766295848232, ('volcanoes form', 'a'): 0.12687550372300352, ('undersea volcanoes', 'a'): 0.12687450372300352, 

In [86]:
def get_keywords_YAKE(text):
    """Extracts keywords from input text using Graph based TextRank algorithm
           @input: (text)-Block of continous text on a single topic 
           @Hyperparameters: ()-None
           @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out={}
    try:
        # Selecting the extractor and loading the input text in it
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=text,language='en', normalization=None)
        
        # It can extract these three types of "phrases" from the input text
        #pos = {'NOUN','VERB','ADJ'}
        #convert_pos = {'VERB': 'v', 'NOUN': 'n','ADJ': 'a'}
        
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist = stopwords.words('english')
        
        keyword_limit = get_keyword_limit(len(text.split()))
        
        extractor.candidate_selection(n=3, stoplist=stoplist)
        extractor.candidate_weighting(window=3,stoplist=stoplist,use_stems=False)
        keyphrases = extractor.get_n_best(n=keyword_limit*2,threshold=0.8)

                
            
        for val in keyphrases:
            out[(val[0])] = val[1]
                
    except:
        out = {}
        traceback.print_exc()

    return out

In [87]:
k = get_keywords_YAKE(text)
print(len(k))
print(k)

28
{'atlantic ridge': 0.04599435489729826, 'divergent plate': 0.05051811913746707, 'plate': 0.05675656522364285, 'divergent plate boundary': 0.0681994202941495, 'crust': 0.0818394401636323, 'volcanoes': 0.08644397137081854, 'divergent': 0.09255724673191565, 'plate boundaries': 0.09885746497827541, 'atlantic ocean': 0.1026581477748343, 'atlantic': 0.10929097903133549, 'molten rock': 0.11286214022530992, 'rock': 0.12285379179720222, 'boundaries': 0.12594121245326878, 'pacific': 0.1275454360674287, 'volcanic activity': 0.1601944782681947, 'molten': 0.17157472445232513, 'called': 0.17436246924346388, 'along': 0.19623912640704033, 'subduction': 0.19820853169223906, 'mid': 0.20116891760408584, 'ridge': 0.20116891760408584, 'oceans': 0.20903296229448917, 'pacific plate': 0.2102683729058276, 'continental crust': 0.2177600078479168, 'pacific ring': 0.2181614702241551, 'lot': 0.22856222633236592, 'activity': 0.22856222633236592, 'volcanoes form': 0.24117553684966264}


In [30]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

In [88]:
pip install keyBERT



  Created wheel for keyBERT: filename=keybert-0.2.0-py3-none-any.whl size=10617 sha256=571899ce64c672ee4f80bb602c52b6c36ea23562672316f07c714a9fb25c99a5
  Stored in directory: c:\users\16692\appdata\local\pip\cache\wheels\ce\ff\82\42c5c7daa648a5bb132941030c36a00e35b713e550e821b58b
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-py3-none-any.whl size=114312 sha256=be32c2dfb6e8f45f4f7e255c500f80dc9f502480d3bdb7d3c3c9acbc09c27d4a
  Stored in directory: c:\users\16692\appdata\local\pip\cache\wheels\28\cb\ae\360fe121dc748add4fabecd46e78c31d1ea2402d341f97e2dc
Successfully built keyBERT sentence-transformers


In [90]:
array_text = []
text_clean = text.replace("\n", " ").replace("\'", "")
array_text.append(text_clean[0:5000])
print(array_text)

['There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is 

In [102]:
def get_keywords_KeyBERT(text):
    """Extracts keywords from input text using Word Embedding based TextRank algorithm
           @input: (text)-Block of continous text on a single topic 
           @Hyperparameters: ()-None
           @Output: (out)-Dictionary with the syntax {('keyword','partofspeech'):weightage}"""
    out={}
    try:
        kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')

        keyword_limit = get_keyword_limit(len(text.split()))

        keywords = kw_extractor.extract_keywords(text, stop_words='english', top_n = keyword_limit*2,keyphrase_ngram_range=(1,2))
    
        for val in keywords:
            out[(val[0])] = val[1]
                
    except:
        out = {}
        traceback.print_exc()

    return out
        

In [103]:
k = get_keywords_KeyBERT(text)
print(len(k))
print(k)

28
{'crust volcanoes': 0.6738, 'erupts volcanoes': 0.671, 'ocean tectonic': 0.6641, 'lined volcanoes': 0.6511, 'forming magma': 0.6431, 'volcanic activity': 0.6322, 'volcanoes form': 0.6299, 'volcanoes forms': 0.6256, 'lot volcanic': 0.6181, 'magma erupts': 0.6082, 'volcanic': 0.5989, 'volcanic arc': 0.5978, 'volcanoes mid': 0.5865, 'volcanoes pacific': 0.5854, 'volcanoes': 0.5847, 'make volcanic': 0.5788, 'volcanoes line': 0.5709, 'line volcanoes': 0.5701, 'tectonic plate': 0.5677, 'molten rock': 0.5611, 'undersea volcanoes': 0.5603, 'tectonic plates': 0.5603, 'ocean crust': 0.5493, 'oceanic crust': 0.5455, 'tectonic': 0.5351, 'boundaries tectonic': 0.5274, 'magma': 0.5178, 'thicker oceanic': 0.5079}
